In [4]:
import os
import sys

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
#from tsdb import pickle_dump
import argparse
from unified_dataloader import UnifiedDataLoader
from saitsd import SAITS
sys.path.append("..")

from data_processing_utils import (
    window_truncate,
    random_mask,
    add_artificial_mask,
    saving_into_h5,
)
from datetime import datetime
import torch.nn as nn
import torch

In [3]:
%cd /content/drive/MyDrive/saits

/content/drive/MyDrive/saits


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from scipy.stats import norm

In [7]:
MODEL_DICT = {
    # Self-Attention (SA) based

    "SAITS": SAITS,
    # RNN based

}

In [8]:




def ornstein_uhlenbeck(x0, theta, mu, sigma, dt, num_steps, num_samples):
    x = np.zeros((num_steps, num_samples))
    x[0] = x0
    for t in range(1, num_steps):
        if t<7:
            x[t] = x[t-1] + theta * (mu - x[t-1]) * dt + sigma * np.sqrt(dt) * np.ones(num_samples)
        else:
            x[t] = x[t-1] + theta * (mu - x[t-1]) * dt + sigma * np.sqrt(dt) * np.random.normal(num_samples)
    return x

# Parameters
x0 = 0  # Initial value
theta = 0.1  # Rate of reversion
mu = 0.5  # Long-term mean
sigma = 0.3  # Volatility
dt = 0.01  # Time step
num_steps = 8  # Number of steps
num_samples =45 # Number of samples at each step

# Simulate Ornstein-Uhlenbeck process
ou_process = ornstein_uhlenbeck(x0, theta, mu, sigma, dt, num_steps, num_samples)




In [9]:
 # volatility
delta_t = 1  # time step size



dt = delta_t
mean = ou_process[6, :] * np.exp(-theta * dt) + mu * (1 - np.exp(-theta * dt))
variance = (sigma**2 / (2 * theta)) * (1 - np.exp(-2 * theta * dt))
ou_process[7, :] = mean + np.random.normal(0, np.sqrt(variance), num_samples)

In [ ]:
empty=np.empty(0)

In [98]:
variance

0.08157116111490817

In [ ]:
num_samples

45

In [ ]:
for i in range(45):
    mean=np.repeat(ou_process[7, i] * np.exp(-theta * dt) + mu * (1 - np.exp(-theta * dt)),45)
    variance = np.repeat((sigma**2 / (2 * theta)) * (1 - np.exp(-2 * theta * dt)),45)

    ou_new=mean + np.random.normal(0, np.sqrt(variance),num_samples)
    empty=np.append(empty,ou_new)


In [ ]:
empty

array([-0.32801771,  0.39889443, -0.24801868, ...,  0.27618894,
        0.16236607,  0.66707857])

In [10]:
df=pd.DataFrame(ou_process)

In [ ]:
result_df = pd.DataFrame()


In [ ]:
for col in df.columns:
    # Duplicate the current column 50 times and add to the result DataFrame
    duplicated_columns = pd.concat([df[col]] * 45, axis=1)
    # Rename the columns to indicate their original name and index
    duplicated_columns.columns = [f'{col}_{i+1}' for i in range(45)]
    # Append to the result DataFrame
    result_df = pd.concat([result_df, duplicated_columns], axis=1)



In [ ]:
result_df = result_df.append(pd.Series(empty, index=result_df.columns), ignore_index=True)




In [11]:
df

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.030500,0.030500,0.030500,0.030500,0.030500,0.030500,0.030500,0.030500,0.030500,0.030500,...,0.030500,0.030500,0.030500,0.030500,0.030500,0.030500,0.030500,0.030500,0.030500,0.030500
2,0.060969,0.060969,0.060969,0.060969,0.060969,0.060969,0.060969,0.060969,0.060969,0.060969,...,0.060969,0.060969,0.060969,0.060969,0.060969,0.060969,0.060969,0.060969,0.060969,0.060969
3,0.091409,0.091409,0.091409,0.091409,0.091409,0.091409,0.091409,0.091409,0.091409,0.091409,...,0.091409,0.091409,0.091409,0.091409,0.091409,0.091409,0.091409,0.091409,0.091409,0.091409
4,0.121817,0.121817,0.121817,0.121817,0.121817,0.121817,0.121817,0.121817,0.121817,0.121817,...,0.121817,0.121817,0.121817,0.121817,0.121817,0.121817,0.121817,0.121817,0.121817,0.121817
5,0.152195,0.152195,0.152195,0.152195,0.152195,0.152195,0.152195,0.152195,0.152195,0.152195,...,0.152195,0.152195,0.152195,0.152195,0.152195,0.152195,0.152195,0.152195,0.152195,0.152195
6,0.182543,0.182543,0.182543,0.182543,0.182543,0.182543,0.182543,0.182543,0.182543,0.182543,...,0.182543,0.182543,0.182543,0.182543,0.182543,0.182543,0.182543,0.182543,0.182543,0.182543
7,0.160219,-0.444650,0.666671,0.615364,0.412741,0.185146,-0.043886,0.600151,0.817910,0.482079,...,0.187284,0.517726,0.353534,0.017573,0.399901,0.181000,0.389982,0.384007,0.320481,0.539550


In [ ]:
from scipy.stats import norm

In [ ]:
norm.fit(ou_process[11])

(0.3280566451599556, 0.3067730689596443)

In [12]:
data=df.T.to_numpy()

In [13]:
data

array([[ 0.        ,  0.0305    ,  0.0609695 ,  0.09140853,  0.12181712,
         0.1521953 ,  0.18254311,  0.16021904],
       [ 0.        ,  0.0305    ,  0.0609695 ,  0.09140853,  0.12181712,
         0.1521953 ,  0.18254311, -0.44465029],
       [ 0.        ,  0.0305    ,  0.0609695 ,  0.09140853,  0.12181712,
         0.1521953 ,  0.18254311,  0.66667084],
       [ 0.        ,  0.0305    ,  0.0609695 ,  0.09140853,  0.12181712,
         0.1521953 ,  0.18254311,  0.61536422],
       [ 0.        ,  0.0305    ,  0.0609695 ,  0.09140853,  0.12181712,
         0.1521953 ,  0.18254311,  0.41274062],
       [ 0.        ,  0.0305    ,  0.0609695 ,  0.09140853,  0.12181712,
         0.1521953 ,  0.18254311,  0.18514559],
       [ 0.        ,  0.0305    ,  0.0609695 ,  0.09140853,  0.12181712,
         0.1521953 ,  0.18254311, -0.04388565],
       [ 0.        ,  0.0305    ,  0.0609695 ,  0.09140853,  0.12181712,
         0.1521953 ,  0.18254311,  0.60015071],
       [ 0.        ,  0.0305    

In [14]:
data=data.reshape(45,8,1)

In [15]:
train_set_X=data[:43]
test_set_X=data[43:44]
val_set_X=data[44:45]

In [16]:
train_set_dict = add_artificial_mask(
        train_set_X, 0.1, "train"
    )
val_set_dict = add_artificial_mask(val_set_X, 0.1, "val")
test_set_dict = add_artificial_mask(
        test_set_X, 0.1, "test"
    )

In [17]:
processed_data = {
        "train": train_set_dict,
        "val":val_set_dict,
        "test": test_set_dict,
    }

In [18]:
saving_dir='/content/sample_data'

In [19]:
saving_into_h5(saving_dir, processed_data, classification_dataset=False)

In [20]:
from configparser import ConfigParser, ExtendedInterpolation

In [21]:
cfg = ConfigParser(interpolation=ExtendedInterpolation())

In [22]:
def read_arguments(arg_parser, cfg_parser):
    # file path
    arg_parser.dataset_base_dir = cfg_parser.get("file_path", "dataset_base_dir")

    # dataset info
    arg_parser.seq_len = cfg_parser.getint("dataset", "seq_len")
    arg_parser.batch_size = cfg_parser.getint("dataset", "batch_size")
    arg_parser.num_workers = cfg_parser.getint("dataset", "num_workers")
    arg_parser.feature_num = cfg_parser.getint("dataset", "feature_num")
    arg_parser.dataset_name = cfg_parser.get("dataset", "dataset_name")
    arg_parser.dataset_path = os.path.join(
        arg_parser.dataset_base_dir, arg_parser.dataset_name
    )
    arg_parser.eval_every_n_steps = cfg_parser.getint("dataset", "eval_every_n_steps")
    # training settings
    arg_parser.MIT = cfg_parser.getboolean("training", "MIT")
    arg_parser.ORT = cfg_parser.getboolean("training", "ORT")
    arg_parser.lr = cfg_parser.getfloat("training", "lr")
    arg_parser.optimizer_type = cfg_parser.get("training", "optimizer_type")
    arg_parser.weight_decay = cfg_parser.getfloat("training", "weight_decay")
    arg_parser.device = cfg_parser.get("training", "device")
    arg_parser.epochs = cfg_parser.getint("training", "epochs")
    arg_parser.early_stop_patience = cfg_parser.getint(
        "training", "early_stop_patience"
    )

    arg_parser.max_norm = cfg_parser.getfloat("training", "max_norm")
    arg_parser.imputation_loss_weight = cfg_parser.getfloat(
        "training", "imputation_loss_weight"
    )
    arg_parser.reconstruction_loss_weight = cfg_parser.getfloat(
        "training", "reconstruction_loss_weight"
    )
    # model settings
    arg_parser.model_name = cfg_parser.get("model", "model_name")
    arg_parser.model_type = cfg_parser.get("model", "model_type")
    return arg_parser

In [24]:

parser = argparse.ArgumentParser()
parser.add_argument("--config_path", type=str, help="path of config file",default="/content/drive/MyDrive/saits/AirQuality_SAITS_best.ini")

_StoreAction(option_strings=['--config_path'], dest='config_path', nargs=None, const=None, default='/content/drive/MyDrive/saits/AirQuality_SAITS_best.ini', type=<class 'str'>, choices=None, required=False, help='path of config file', metavar=None)

In [25]:
args,unknown = parser.parse_known_args()

In [26]:
args

Namespace(config_path='/content/drive/MyDrive/saits/AirQuality_SAITS_best.ini')

In [27]:
cfg.read(args.config_path)

['/content/drive/MyDrive/saits/AirQuality_SAITS_best.ini']

In [28]:
args = read_arguments(args, cfg)

In [29]:
args

Namespace(config_path='/content/drive/MyDrive/saits/AirQuality_SAITS_best.ini', dataset_base_dir='generated_datasets', seq_len=8, batch_size=100, num_workers=4, feature_num=1, dataset_name='AirQuality_seqlen24_01masked', dataset_path='generated_datasets/AirQuality_seqlen24_01masked', eval_every_n_steps=7, MIT=True, ORT=True, lr=0.0008821387950693266, optimizer_type='adam', weight_decay=0.0, device='cuda', epochs=10000, early_stop_patience=30, max_norm=0.0, imputation_loss_weight=1.0, reconstruction_loss_weight=1.0, model_name='AirQuality_SAITS_best', model_type='SAITS')

In [30]:
if args.model_type in ["Transformer", "SAITS"]:  # if SA-based model
        args.input_with_mask = cfg.getboolean("model", "input_with_mask")
        args.n_groups = cfg.getint("model", "n_groups")
        args.n_group_inner_layers = cfg.getint("model", "n_group_inner_layers")
        args.param_sharing_strategy = cfg.get("model", "param_sharing_strategy")
        assert args.param_sharing_strategy in [
            "inner_group",
            "between_group",
        ], 'only "inner_group"/"between_group" sharing'
        args.d_model = cfg.getint("model", "d_model")
        args.d_inner = cfg.getint("model", "d_inner")
        args.n_head = cfg.getint("model", "n_head")
        args.d_k = cfg.getint("model", "d_k")
        args.d_v = cfg.getint("model", "d_v")
        args.dropout = cfg.getfloat("model", "dropout")
        args.diagonal_attention_mask = cfg.getboolean(
            "model", "diagonal_attention_mask"
        )

In [31]:
dict_args = vars(args)

In [32]:
model_args = {
            "device":'cpu',
            "MIT": args.MIT,
            # imputer args
            "n_groups": dict_args["n_groups"],
            "n_group_inner_layers": args.n_group_inner_layers,
            "d_time": args.seq_len,
            "d_feature": args.feature_num,
            "dropout": dict_args["dropout"],
            "d_model": dict_args["d_model"],
            "d_inner": dict_args["d_inner"],
            "n_head": dict_args["n_head"],
            "d_k": dict_args["d_k"],
            "d_v": dict_args["d_v"],
            "input_with_mask": args.input_with_mask,
            "diagonal_attention_mask": args.diagonal_attention_mask,
            "param_sharing_strategy": args.param_sharing_strategy,
        }

In [33]:
time_now = datetime.now().__format__("%Y-%m-%d_T%H:%M:%S")

In [34]:
unified_dataloader = UnifiedDataLoader(
        saving_dir,
        args.seq_len,
        1,
        args.model_type,
        args.batch_size,
        args.num_workers,
        args.MIT,
    )

In [35]:
train_dataloader,val_dataloader= unified_dataloader.get_train_val_dataloader()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [36]:
train_dataloader

In [90]:
model = MODEL_DICT[args.model_type](**model_args)
OPTIMIZER = {"adam": torch.optim.Adam, "adamw": torch.optim.AdamW}

In [91]:
indices, X, missing_mask, X_holdout, indicating_mask = map(
                    lambda x: x.to(args.device),next(iter(train_dataloader))
                )

In [82]:
inputs = {
                    "indices": indices,
                    "X": X,
                    "missing_mask": missing_mask,
                    "X_holdout": X_holdout,
                    "indicating_mask": indicating_mask,
                }

In [81]:
def result_processing(results):
    """process results and losses for each training step"""
    results["total_loss"] = torch.tensor(0.0, device=args.device)
    if args.MIT:
        results["total_loss"] += results["reconstruction_loss"]
    return results

In [83]:
def process_each_training_step(
    results, optimizer
):
    """process each training step and return whether to early stop"""

    # apply gradient clipping if args.max_norm != 0
    if args.max_norm != 0:
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=args.max_norm)
    results["total_loss"].backward()
    optimizer.step()
    return False

In [84]:
def model_processing(
    data,
    model,
    stage,
    # following arguments are only required in the training stage
    optimizer=None,
    val_dataloader=None,

    training_controller=None,
    logger=None,
):
    if stage == "train":
        optimizer.zero_grad()
        if not args.MIT:
              # then for self-attention based models, i.e. Transformer/SAITS
                indices, X, missing_mask = map(lambda x: x.to(args.device), data)
                inputs = {"indices": indices, "X": X, "missing_mask": missing_mask}
                results = result_processing(model(inputs, stage))
                early_stopping = process_each_training_step(
                results,
                optimizer,
                val_dataloader,
                training_controller,

            )
        else:


                indices, X, missing_mask, X_holdout, indicating_mask = map(
                    lambda x: x.to(args.device), data
                    )

                inputs = {"indices": indices, "X": X, "missing_mask": missing_mask,"X_holdout":X_holdout,"indicating_mask":indicating_mask,"idx":7}

                results = result_processing(model(inputs, stage))

                early_stopping = process_each_training_step(
                    results,
                    optimizer
                    )
        return early_stopping
    return inputs, results




In [94]:
def train(
    model,
    optimizer,
    train_dataloader
):
    for epoch in range(2000):
        early_stopping = False
        args.final_epoch = True if epoch == args.epochs - 1 else False
        for idx, data in enumerate(train_dataloader):
            model.train()
            early_stopping = model_processing(
                data,
                model,
                "train",
                optimizer
            )

In [92]:
model.to(args.device)

SAITS(
  (layer_stack_for_first_block): ModuleList(
    (0): EncoderLayer(
      (layer_norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (slf_attn): MultiHeadAttention(
        (w_qs): Linear(in_features=32, out_features=512, bias=False)
        (w_ks): Linear(in_features=32, out_features=512, bias=False)
        (w_vs): Linear(in_features=32, out_features=256, bias=False)
        (attention): ScaledDotProductAttention(
          (dropout): Dropout(p=0, inplace=False)
        )
        (fc): Linear(in_features=256, out_features=32, bias=False)
      )
      (dropout): Dropout(p=0.0, inplace=False)
      (pos_ffn): PositionWiseFeedForward(
        (w_1): Linear(in_features=32, out_features=32, bias=True)
        (w_2): Linear(in_features=32, out_features=32, bias=True)
        (layer_norm): LayerNorm((32,), eps=1e-06, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (dropout): Dropout(p=0.0, inplace=False)
  (position_enc): P

In [87]:
optimizer = OPTIMIZER[args.optimizer_type](
            model.parameters(), lr=dict_args["lr"], weight_decay=args.weight_decay
        )

In [95]:
train(
            model,
            optimizer,
            train_dataloader
        )



Exception ignored in: <function _after_fork at 0x78ad24641c60>
Traceback (most recent call last):
Exception ignored in:   File "/usr/lib/python3.10/threading.py", line 1622, in _after_fork
    threads.update(_dangling)
  File "/usr/lib/python3.10/_weakrefset.py", line 66, in __iter__
Exception ignored in: <function _after_fork at 0x78ad24641c60>Exception ignored in: 
<function _after_fork at 0x78ad24641c60><function _after_fork at 0x78ad24641c60>Traceback (most recent call last):


  File "/usr/lib/python3.10/threading.py", line 1635, in _after_fork
    Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1635, in _after_fork
Traceback (most recent call last):
    thread._reset_internal_locks(False)    item = itemref()

  File "/usr/lib/python3.10/threading.py", line 1622, in _after_fork
thread._reset_internal_locks(False)  File "/usr/lib/python3.10/threading.py", line 885, in _reset_internal_locks
KeyboardInterrupt
        KeyboardInterrupt: : threads.upd

KeyboardInterrupt: 

In [96]:
model.eval()
with torch.no_grad():
  print(model.impute(inputs))

(tensor([11.4684, 11.4684, 11.4684, 11.4684, 11.4684, 11.4684, 11.4684, 11.4684,
        11.4684, 11.4684, 11.4684, 11.4684, 11.4684, 11.4684, 11.4684, 11.4684,
        11.4684, 11.4684, 11.4684, 11.4684, 11.4684, 11.4684, 11.4684, 11.4684,
        11.4684, 11.4684, 11.4684, 11.4684, 11.4684, 11.4684, 11.4684, 11.4684,
        11.4684, 11.4684, 11.4684, 11.4684, 11.4684, 11.4684, 11.4684, 11.4684,
        11.4684, 11.4684, 11.4684], device='cuda:0'), tensor([3.8755e-07, 3.8755e-07, 3.8755e-07, 3.8755e-07, 3.8755e-07, 3.8755e-07,
        3.8755e-07, 3.8755e-07, 3.8755e-07, 3.8755e-07, 3.8755e-07, 3.8755e-07,
        3.8755e-07, 3.8755e-07, 3.8755e-07, 3.8755e-07, 3.8755e-07, 3.8755e-07,
        3.8755e-07, 3.8755e-07, 3.8755e-07, 3.8755e-07, 3.8755e-07, 3.8755e-07,
        3.8755e-07, 3.8755e-07, 3.8755e-07, 3.8755e-07, 3.8755e-07, 3.8755e-07,
        3.8755e-07, 3.8755e-07, 3.8755e-07, 3.8755e-07, 3.8755e-07, 3.8755e-07,
        3.8755e-07, 3.8755e-07, 3.8755e-07, 3.8755e-07, 3.8755e-0

In [71]:
model.impute(inputs)

(tensor([-98.5634, -98.5634, -98.5634, -98.5634, -98.5634, -98.5634, -98.5634,
         -98.5634, -98.5634, -98.5634, -98.5634, -98.5634, -98.5634, -98.5634,
         -98.5634, -98.5634, -98.5634, -98.5634, -98.5634, -98.5634, -98.5634,
         -98.5634, -98.5634, -98.5634, -98.5634, -98.5634, -98.5634, -98.5634,
         -98.5634, -98.5634, -98.5634, -98.5634, -98.5634, -98.5634, -98.5634,
         -98.5634, -98.5634, -98.5634, -98.5634, -98.5634, -98.5634, -98.5634,
         -98.5634], device='cuda:0', grad_fn=<SelectBackward0>),
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        device='cuda:0', grad_fn=<SoftplusBackward0>))

In [66]:
model.impute(inputs)

(tensor([-52.1960, -52.1960, -52.1960, -52.1960, -52.1960, -52.1960, -52.1960,
         -52.1960, -52.1960, -52.1960, -52.1960, -52.1960, -52.1960, -52.1960,
         -52.1960, -52.1960, -52.1960, -52.1960, -52.1960, -52.1960, -52.1960,
         -52.1960, -52.1960, -52.1960, -52.1960, -52.1960, -52.1960, -52.1960,
         -52.1960, -52.1960, -52.1960, -52.1960, -52.1960, -52.1960, -52.1960,
         -52.1960, -52.1960, -52.1960, -52.1960, -52.1960, -52.1960, -52.1960,
         -52.1960], device='cuda:0', grad_fn=<SelectBackward0>),
 tensor([3.1697e-42, 3.1697e-42, 3.1697e-42, 3.1697e-42, 3.1697e-42, 3.1697e-42,
         3.1697e-42, 3.1697e-42, 3.1697e-42, 3.1697e-42, 3.1697e-42, 3.1697e-42,
         3.1697e-42, 3.1697e-42, 3.1697e-42, 3.1697e-42, 3.1697e-42, 3.1697e-42,
         3.1697e-42, 3.1697e-42, 3.1697e-42, 3.1697e-42, 3.1697e-42, 3.1697e-42,
         3.1697e-42, 3.1697e-42, 3.1697e-42, 3.1697e-42, 3.1697e-42, 3.1697e-42,
         3.1697e-42, 3.1697e-42, 3.1697e-42, 3.1697e-42,